In [7]:
import numpy as np
from datetime import datetime


In [3]:
# B-tree

# Create a node
class BTreeNode:
    def __init__(self, leaf=False):
        self.leaf = leaf
        self.keys = []
        self.child = []

# Tree
class FileExplorer:
    def __init__(self, t):
        self.root = BTreeNode(True)
        self.t = t

    # Insert a key
    def insert(self, k):                            # send in the tupple k (k[0] is the key, k[1] is the address that store the key)
        root = self.root                            # root node reference
        if len(root.keys) == (2 * self.t) - 1:      # in case of the root is full
            temp = BTreeNode()                      # create new node
            self.root = temp                        # Set the new node as new root
            temp.child.insert(0, root)              # set the old root node as the first child of the new root node
            self.split_child(temp, 0)               # split the old root node and insert its middle key into new root node
            self.insert_non_full(temp, k)           # insert k into the new tree
        else:
            self.insert_non_full(root, k)           # else just insert the k into the tree

    # Insert non full
    def insert_non_full(self, x, k):
        i = len(x.keys) - 1
        if x.leaf:                                  # in case of x is a leaf node
            x.keys.append((None, None))             # Make room for a new key
            while i >= 0 and k[0] < x.keys[i][0]:
                x.keys[i + 1] = x.keys[i]           # Shift keys to the right
                i -= 1
            x.keys[i + 1] = k                       # insert key at the right position
        else:                                       # in case x is not a leaf node
            while i >= 0 and k[0] < x.keys[i][0]:   # Find the right x.child to insert the key
                i -= 1
            i += 1
            if len(x.child[i].keys) == (2 * self.t) - 1:    # if the targetted child is full
                self.split_child(x, i)                      # split it
                if k[0] > x.keys[i][0]:                     # After splitting, compare k with the middle key that moved up to decide whether to insert into the left or right child.
                    i += 1
            self.insert_non_full(x.child[i], k)             # Recursively insert k into the appropriate child

    # Split the child
    def split_child(self, x, i):            # x is the parent node, i is the child index of x that needs splitting
        t = self.t                          # minimum degree of B-tree
        y = x.child[i]                      # Get the full child y that needs to be split
        z = BTreeNode(y.leaf)               # create new node that will hold half of y
        x.child.insert(i + 1, z)            # insert node z as the child of x
        x.keys.insert(i, y.keys[t - 1])     # moves y middle key to x
        z.keys = y.keys[t: (2 * t) - 1]     # z gets the second half keys of y
        y.keys = y.keys[0: t - 1]           # y keeps the first half keys
        if not y.leaf:                      # in case y is internal node
            z.child = y.child[t: 2 * t]     # y keeps the first t children
            y.child = y.child[0: t]         # z gets the remaining children of y

    # Delete a node
    def delete(self, x, k):
        t = self.t
        i = 0
        while i < len(x.keys) and k[0] > x.keys[i][0]:      # Move to the node that contains key k
            i += 1
        if x.leaf:
            #                                               # Case 1: Node is a leaf
            if i < len(x.keys) and x.keys[i][0] == k[0]:    # Simply delete it from the key list
                x.keys.pop(i)
                return
        else:
            #                                               # Case 2: Key is found in an internal node
            if i < len(x.keys) and x.keys[i][0] == k[0]:
                return self.delete_internal_node(x, k, i)
            #                                               # Case 3: Key is not in node, go to the proper child
            if len(x.child[i].keys) < t:                    # Ensure child has >= t keys
                self.fill(x, i)                             # if not the we call fill function
            self.delete(x.child[i], k)                      # recursively call the delete function on the subtree

    def delete_internal_node(self, x, k, i):
        t = self.t
        #                                                   # Case 2a: Predecessor has enough keys
        if len(x.child[i].keys) >= t:
            pred_key = self.get_predecessor(x, i)
            x.keys[i] = pred_key                            # Replace the key k of x with its predecessor from the left subtree
            self.delete(x.child[i], pred_key)               # Delete the predecessor key from the left child
        #                                                   # Case 2b: Successor has enough keys
        elif len(x.child[i + 1].keys) >= t:
            succ_key = self.get_successor(x, i)
            x.keys[i] = succ_key                            # Replace the key k of x with its successor from the right subtree
            self.delete(x.child[i + 1], succ_key)           # Delete the successor key from the right child
        #                                                   # Case 2c: Both children have fewer than t keys
        else:
            self.merge(x, i)                                # merge
            self.delete(x.child[i], k)

    def get_predecessor(self, x, i):                    # Traverse left subtree to find the max key
        cur = x.child[i]
        while not cur.leaf:
            cur = cur.child[len(cur.child) - 1]
        return cur.keys[len(cur.keys) - 1]

    def get_successor(self, x, i):                      # Traverse right subtree to find the min key
        cur = x.child[i + 1]
        while not cur.leaf:
            cur = cur.child[0]
        return cur.keys[0]

    def merge(self, x, i):                      # Merge function to merge two children
        t = self.t
        child = x.child[i]
        sibling = x.child[i + 1]
        child.keys.append(x.keys[i])            # Merge key i from x to child
        child.keys.extend(sibling.keys)         # Append sibling's keys to child
        if not child.leaf:                      # If sibling has children, append them to child
            child.child.extend(sibling.child)
        x.keys.pop(i)                           # Remove the key from x
        x.child.pop(i + 1)                      # Delete sibling from child list
        if len(x.keys) == 0:                    # If root becomes empty, reduce the height of the tree
            self.root = child

    def fill(self, x, i):                       # Fill function to ensure child has at least t keys
        t = self.t
        if i != 0 and len(x.child[i - 1].keys) >= t:                    # Borrow from the previous sibling
            self.borrow_from_prev(x, i)
        elif i != len(x.child) - 1 and len(x.child[i + 1].keys) >= t:   # Borrow from the next sibling
            self.borrow_from_next(x, i)
        else:                                                           # Merge with sibling
            if i != len(x.child) - 1:
                self.merge(x, i)
            else:
                self.merge(x, i - 1)

    def borrow_from_prev(self, x, i):
        child = x.child[i]
        sibling = x.child[i - 1]

        child.keys.insert(0, x.keys[i - 1])             # Move the key i from x to first of child
        x.keys[i - 1] = sibling.keys.pop()              # Move the last key from sibling to x
        if not child.leaf:                              # Move the last child of sibling to child if sibling is not a leaf
            child.child.insert(0, sibling.child.pop())

    def borrow_from_next(self, x, i):
        child = x.child[i]
        sibling = x.child[i + 1]

        child.keys.append(x.keys[i])                    # Move the key i from x to last of child
        x.keys[i] = sibling.keys.pop(0)                 # Move the first key from sibling to x
        if not child.leaf:                              # Move the first child of sibling to child if sibling is not a leaf
            child.child.append(sibling.child.pop(0))

#     # Print the tree
#     def print_tree(self, x, l=0):
#         print("Level ", l, " ", len(x.keys), end=":")
#         for i in x.keys:
#             print(i, end=" ")
#         print()
#         l += 1
#         if len(x.child) > 0:
#             for i in x.child:
#                 self.print_tree(i, l)

# # Example usage
# B = FileExplorer(2)
# for i in range(10):
#     B.insert((i, 2 * i))
# B.print_tree(B.root)
# B.delete(B.root, (8,))
# print("\n")
# B.print_tree(B.root)


In [22]:
class FileSystemNode:
    def __init__(self, name, node_type, t):
        self.name = name                    # file and folder name
        self.type = node_type               # "file" or "folder"
        self.created_at = datetime.now()    # the date of file or folder creation

        if node_type == "folder":           # if folder, initiate child (B-Tree)
            self.children = FileExplorer(t)
        else:
            self.children = None

class FileSystem:
    def __init__(self, t):
        self.tree = FileExplorer(t)     # Creates a B-Tree (FileExplorer) to manage all files and folders
        self.t = t                      # Minimum degree t
        # self.root = BTreeNode(True)     # Creates a root node of the B-Tree
        # self.tree.root = self.root      # Assigns the newly created root node to the FileExplorer's root
        self.tree.root = BTreeNode(True)
        root_folder = FileSystemNode("root", "folder", self.t)      # Creates the actual "root" folder of your file system
        self.tree.insert(("root", root_folder))                     # Insert the "root" folder into the file system B-Tree so it can serve as the base of all other folders and files.
        #                                                           # It maps the string "root" to the FileSystemNode object that holds folder content.

    def _path_str(self, path): # Dat
        if path:
           return '/'.join(path)
        return 'root'

    def _find_node(self, path): # Dat
        current = self.tree.root.keys[0][1].children.root   # starts from the "root" folder’s children
        node = None                                         # Initialize a dummy to support loop
        for segment in path:                                # loop through each segment of the path
            found = False
            for i in range(len(current.keys)):              #
                if current.keys[i][0] == segment and current.keys[i][1].type == "folder":       # loop through the current folder to find the next segment
                    node = current.keys[i][1]                                                   # go deeper
                    current = node.children.root                                                # update current to that folder children
                    found = True
                    break
            if not found:                                   # if the segment not found
                return None                                 # the path is invalid
        return node

    def _getParentNode(self, path):
        if path:
            parent_node = self._find_node(path)
        else:
            parent_node = self.tree.root.keys[0][1]

        if not parent_node or parent_node.type != "folder":
            print(f"\033[91m[Error] Path '{self._path_str(path)}' does not exist or is not a folder\033[0m")
            return None
        return parent_node

    def create_folder(self, folder_name, path=[]):

        parent_node = self._getParentNode(path)
        if not parent_node:
            return

        for key, node in parent_node.children.root.keys:
            if key == folder_name:
                print(f"\033[91m[ERROR] A folder or file named '{folder_name}' already exists in '{self._path_str(path)}'\n\033[0m")
                return
        parent_node.children.insert((folder_name, FileSystemNode(folder_name, "folder", self.t)))
        print(f"\033[34m[INFO] Folder '{folder_name}' created in {self._path_str(path)}\n\033[0m")

    def create_file(self, file_name, path=[]):

        parent_node = self._getParentNode(path)
        if not parent_node:
            return

        for key, node in parent_node.children.root.keys:
            if key == file_name:
                print(f"\033[91m [ERROR] A file named '{file_name}' already exists in '{self._path_str(path)}'\n\033[0m")
                return

        parent_node.children.insert((file_name, FileSystemNode(file_name, "file", self.t)))
        print(f"\033[34m[INFO] File '{file_name}' created in {self._path_str(path)}\n\033[0m")

    def _recursive_delete(self, node):
        if node.type != "folder":                       # Recursive delete if the type is folder
            return

        btree = node.children.root                      # store the subtree of the node
        for i in range(len(btree.keys) - 1, -1, -1):    # Iterate in reverse to safely delete
            key, child_node = btree.keys[i]
            if child_node.type == "folder":
                self._recursive_delete(child_node)
            # Delete the file or folder from the current B-Tree
            node.children.delete(node.children.root, (key))

    def delete_file(self, name, path=[]):
        # Find the parent folder node
        parent_node = self._getParentNode(path)
        current_btree = parent_node.children.root

        # Find the File to delete
        found = False
        for i in range(len(current_btree.keys)):
            key, node = current_btree.keys[i]
            if key == name and node.type == "file":
                # Delete this file from the B-Tree
                parent_node.children.delete(parent_node.children.root, (name,))
                print(f" \033[34m[INFO] Delete file '{name}' from {self._path_str(path)}\n\033[0m")
                found = True
                break

        if not found:
            print(f"\033[91m[Error]: File '{name}' not found in {self._path_str(path)}\033[0m")

    def delete_folder(self, name, path=[]):
        # Step 1: Navigate to the parent folder
        parent_node = self._getParentNode(path)
        current_btree = parent_node.children.root
        target_folder_node = None

        # Step 2: Find the folder to delete inside the parent
        for i in range(len(current_btree.keys)):
            key, node = current_btree.keys[i]
            if key == name and node.type == "folder":
                target_folder_node = node
                break

        if not target_folder_node:
            print(f"\033[91m[Error]: Folder '{name}' not found in {self._path_str(path)}\033[0m")
            return

        # Step 3: Recursively delete contents
        self._recursive_delete(target_folder_node)

        # Step 4: Delete the folder node itself from the B-Tree
        parent_node.children.delete(parent_node.children.root, (name,))
        print(f"\033[34m[INFO] Delete folder '{name}' from {self._path_str(path)}\n\033[0m")

    def rename_node(self, old_name, new_name, path=[]):

        parent_node = self._getParentNode(path)
        current_btree = parent_node.children.root

        for existing_key, _ in current_btree.keys:
            if existing_key == new_name:
                print(f"\033[91m[Error]: '{new_name}' already exists in {self._path_str(path)}\033[0m")
                return

        for i in range(len(current_btree.keys)):
            key, node = current_btree.keys[i]
            if key == old_name:
                parent_node.children.delete(parent_node.children.root, (old_name, node))
                node.name = new_name
                parent_node.children.insert((new_name, node))
                print(f"\033[34m[INFO] Renamed '{old_name}' to '{new_name}' in {self._path_str(path)}\033[0m")
                return

        print(f"\033[91m[Error]: '{old_name}' not found in {self._path_str(path)}\033[0m")

    def move_file(self, file_name, source_path=[], dest_path=[]):
        # Find the source parent folder node
        source_parent = self._find_node(source_path) if source_path else self.tree.root.keys[0][1]
        if not source_parent or source_parent.type != "folder":
            print(f"\033[91m[Error]: Source path {self._path_str(source_path)} does not exist or is not a folder\033[0m")
            return

        # Find the destination parent folder node
        dest_parent = self._find_node(dest_path) if dest_path else self.tree.root.keys[0][1]
        if not dest_parent or dest_parent.type != "folder":
            print(f"\033[91m[Error]: Destination path {self._path_str(dest_path)} does not exist or is not a folder\033[0m")
            return

        # Find the file to move inside the source folder
        file_node = None
        for i, (key, node) in enumerate(source_parent.children.root.keys):
            if key == file_name and node.type == "file":
                file_node = node
                break
        if not file_node:
            print(f"\033[91m[Error]: File '{file_name}' not found in {self._path_str(source_path)}\033[0m")
            return

        # Check for name conflict in the destination folder
        for key, node in dest_parent.children.root.keys:
            if key == file_name:
                print(f"\033[91m[Error]: '{file_name}' already exists in {self._path_str(dest_path)}\033[0m")
                return

        # Delete the file from the source folder
        source_parent.children.delete(source_parent.children.root, (file_name,))

        # Insert the file into the destination folder
        dest_parent.children.insert((file_name, file_node))
        print(f"\033[34m[INFO] Moved file '{file_name}' from {self._path_str(source_path)} to {self._path_str(dest_path)}\033[0m")

    def move_folder(self, folder_name, source_path=[], dest_path=[]):
        # Find the source parent folder node
        source_parent = self._getParentNode(source_path)

        # Find the destination parent folder node
        dest_parent = self._getParentNode(dest_path)

        # Find the folder to move inside the source folder
        folder_node = None
        for i, (key, node) in enumerate(source_parent.children.root.keys):
            if key == folder_name and node.type == "folder":
                folder_node = node
                break
        if not folder_node:
            print(f"\033[91m[Error]: Folder '{folder_name}' not found in {self._path_str(source_path)}\033[0m")
            return

        # Check for name conflict in the destination folder
        for key, node in dest_parent.children.root.keys:
            if key == folder_name:
                print(f"\033[91m[Error]: Folder '{folder_name}' already exists in {self._path_str(dest_path)}\033[0m")
                return

        # Delete the folder from the source folder
        source_parent.children.delete(source_parent.children.root, (folder_name,))

        # Insert the folder into the destination folder
        dest_parent.children.insert((folder_name, folder_node))
        print(f"\033[34m[INFO] Moved folder '{folder_name}' from {self._path_str(source_path)} to {self._path_str(dest_path)}\033[0m")

    def _display(self, node, prefix=""):
        total = len(node.keys)
        for i, (name, node) in enumerate(node.keys):
            connector = "└── " if i == total - 1 else "├── "
            node_type = node.type
            created_at = node.created_at.strftime("%Y-%m-%d %H:%M:%S")

            print(f"{prefix}{connector}{name} ({node_type}) - Created at: {created_at}")

            if node_type == "folder":
                extension = "    " if i == total - 1 else "│   "
                self._display(node.children.root, prefix + extension)

    def display_tree(self):
        print("\n\033[34m==== [File System Structure]====\n")
        root_node = self.tree.root.keys[0][1]
        print("root/")  # In dòng gốc trước
        self._display(root_node.children.root, prefix="")
        print("\033[0m\n")

    def _search_recursive(self, node, target_name, current_path, results, target_type):
        """
        Recursively search for target_name of target_type ("file" or "folder") under the current node.
        current_path: list of folder names leading to current node
        results: list to store tuples (found_node, full_path)
        """
        for key, fsnode in node.keys:
            path_now = current_path + [key]
            if key == target_name and fsnode.type == target_type:
                results.append((fsnode, path_now))

            # If it's a folder, continue searching recursively inside it
            if fsnode.type == "folder":
                self._search_recursive(fsnode.children.root, target_name, path_now, results, target_type)

    def search_file(self, file_name):
        results = []
        root_node = self.tree.root.keys[0][1]  # root folder node
        self._search_recursive(root_node.children.root, file_name, ["root"], results, "file")

        if not results:
            print(f" [INFO] No file named '{file_name}' found.")
        else:
            print(f" [INFO] Search results for file '{file_name}':")
            for fsnode, path in results:
                created = fsnode.created_at.strftime("%Y-%m-%d %H:%M:%S")
                print(f" - {created} | Path: {'/'.join(path)}")

    def search_folder(self, folder_name):
        results = []
        root_node = self.tree.root.keys[0][1]  # root folder node
        self._search_recursive(root_node.children.root, folder_name, ["root"], results, "folder")

        if not results:
            print(f" [INFO] No folder named '{folder_name}' found.")
        else:
            print(f" [INFO] Search results for folder '{folder_name}':")
            for fsnode, path in results:
                created = fsnode.created_at.strftime("%Y-%m-%d %H:%M:%S")
                print(f" - {created} | Path: {'/'.join(path)}")


    def menu(self):
        print("="*45)
        print("** FILE SYSTEM MENU **".center(42))
        print("="*45)

        box_width = 43
        top_bottom = "+" + "-" * box_width + "+"
        menu_text = (
            top_bottom + "\n" +
            "| {:<20} {:<20} |\n".format("A. Create folder", "B. Create file") +
            "| {:<20} {:<20} |\n".format("C. Delete folder", "D. Delete file") +
            "| {:<20} {:<20} |\n".format("E. Rename folder", "F. Rename file") +
            "| {:<20} {:<20} |\n".format("G. Move folder", "H. Move file") +
            "| {:<20} {:<20} |\n".format("I. Search folder", "J. Search file") +
            "| {:<20} {:<20} |\n".format("K. Display File Explorer", "") +
            "| {:<20} {:<20} |\n".format("M. Menu","L. Exit") +
            top_bottom
        )
        print(menu_text)

        while True:
            choice = input("\n-> Enter your choice: ").strip().upper()

            if choice == "A":
                folder_name = input("[INPUT] Folder name: ").strip()
                path = input("[INPUT] Path (e.g., projects/python) [leave empty for root]: ")
                path_list = path.split("/") if path else []
                self.create_folder(folder_name, path_list)
                print("\n")

            elif choice == "B":
                file_name = input("[INPUT] File name: ").strip()
                path = input("[INPUT] Path: ")
                path_list = path.split("/") if path else []
                self.create_file(file_name, path_list)
                print("\n")

            elif choice == "C":
                folder_name = input("[INPUT] Folder name to delete: ")
                path = input("[INPUT] Parent path: ")
                path_list = path.split("/") if path else []
                self.delete_folder(folder_name, path_list)
                print("\n")

            elif choice == "D":
                file_name = input("[INPUT] File name to delete: ")
                path = input("[INPUT] Parent path: ")
                path_list = path.split("/") if path else []
                self.delete_file(file_name, path_list)
                print("\n")

            elif choice == "E":
                old_name = input("[INPUT] Old folder name: ")
                new_name = input("[INPUT] New folder name: ")
                path = input("[INPUT] Parent path: ")
                path_list = path.split("/") if path else []
                self.rename_node(old_name, new_name, path_list)
                print("\n")

            elif choice == "F":
                old_name = input("[INPUT] Old file name: ")
                new_name = input("[INPIT] New file name: ")
                path = input("[INPUT] Parent path: ")
                path_list = path.split("/") if path else []
                self.rename_node(old_name, new_name, path_list)
                print("\n")

            elif choice == "G":
                folder_name = input("[INPUT] Folder name to move: ")
                source_path = input("[INPUT] Source path (ex: projects/python) [leave empty for root]: ")
                dest_path = input("[INPUT] Destination path (ex: documents) [leave empty for root]: ")
                source_path_list = source_path.split("/") if source_path else []
                dest_path_list = dest_path.split("/") if dest_path else []
                self.move_folder(folder_name, source_path_list, dest_path_list)
                print("\n")

            elif choice == "H":
                file_name = input("[INPUT] Enter file name to move: ")
                source_path = input("[INPUT]  Enter source path (ex: projects/python) [leave empty for root]: ")
                dest_path = input("[INPUT] Enter destination path (ex: documents) [leave empty for root]: ")
                source_path_list = source_path.split("/") if source_path else []
                dest_path_list = dest_path.split("/") if dest_path else []
                self.move_file(file_name, source_path_list, dest_path_list)
                print("\n")

            elif choice == "I":
                folder_name = input("[INPUT] Enter folder name to search: ")
                self.search_folder(folder_name)
                print("\n")

            elif choice == "J":
                file_name = input("[INPUT] Enter file name to search: ")
                self.search_file(file_name)
                print("\n")

            elif choice == "K":
                print("\n")
                self.display_tree()

            elif choice == "L":
                print("\n ...Exiting the file system")
                break


            elif choice == "M":
                print(menu_text)

            else:
                print("\033[91m[Error] Invalid choice. Please try again.\033[0m")

            print(end="")




In [23]:
fs = FileSystem(t=6)

fs.create_folder("Hololive")
fs.create_folder("Myth", ["Hololive"])
fs.create_folder("Kiara", ["Hololive","Myth"])
# fs.create_folder("Journey")
# fs.create_file("Path1", ["Journey"])
# fs.create_file("Path2", ["Journey"])
# fs.create_file("Path3", ["Journey"])
# fs.create_folder("Path4", ["Journey"])
# fs.create_file("Path5", ["Journey"])
# fs.create_file("Congratulations!!",["Journey", "Path4"])
fs.create_folder("Gura", ["Hololive","Myth"])
fs.create_file("Chumbuds", ["Hololive","Myth", "Gura"])
fs.create_file("Mori", ["Hololive","Myth"])
fs.create_file("Ame",["Hololive","Myth"])
fs.create_file("Ina", ["Hololive","Myth"])
fs.create_folder("Council",["Hololive"])
fs.create_file("Kronii",["Hololive","Council"])
fs.create_file("Mumei",["Hololive","Council"])
fs.create_file("Fauna",["Hololive","Council"])
fs.create_file("Baelz",["Hololive","Council"])
fs.create_file("Sana",["Hololive","Council"])
fs.create_folder("V_Singer",["Hololive"])
fs.create_file("Irys", ["Hololive", "V_Singer"])

dummy1 = fs.tree.root.keys[0][1]

print(dummy1.children.root.keys[0][1].children.root.keys[2][1].children.root.keys[0][0])

fs.menu()



[INFO] Folder 'Hololive' created in root

[INFO] Folder 'Myth' created in Hololive

[INFO] Folder 'Kiara' created in Hololive/Myth

[INFO] Folder 'Gura' created in Hololive/Myth

[INFO] File 'Chumbuds' created in Hololive/Myth/Gura

[INFO] File 'Mori' created in Hololive/Myth

[INFO] File 'Ame' created in Hololive/Myth

[INFO] File 'Ina' created in Hololive/Myth

[INFO] Folder 'Council' created in Hololive

[INFO] File 'Kronii' created in Hololive/Council

[INFO] File 'Mumei' created in Hololive/Council

[INFO] File 'Fauna' created in Hololive/Council

[INFO] File 'Baelz' created in Hololive/Council

[INFO] File 'Sana' created in Hololive/Council

[INFO] Folder 'V_Singer' created in Hololive

[INFO] File 'Irys' created in Hololive/V_Singer

Irys
          ** FILE SYSTEM MENU **          
+-------------------------------------------+
| A. Create folder     B. Create file       |
| C. Delete folder     D. Delete file       |
| E. Rename folder     F. Rename file       |
| G. Move folder 